In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import pandas as pd
import numpy as np
import httpx
import requests
from src.moex_api import get_market_candles, make_url, make_query, get, get_market_candles

## fetch

In [3]:
def sequential_moex_fetch(request_url = 'https://iss.moex.com/iss/securities.json', datakey = 'securities', args_dict = None):
    df_hist = []
    df = pd.DataFrame()
    init = True
    pbar = tqdm()
    if args_dict is not None:
        arguments = {k: v for k, v in args_dict.items()}
    else:
        arguments = {'start': 0}
    with requests.Session() as session:
        while init or not df.empty:
            if init:
                init = False
            params = "&".join([f"{k}={v}" for k, v in arguments.items()])
            response = session.get(f"{request_url}?{params}").json()
            df = pd.DataFrame(response[datakey]['data'], columns = response[datakey]['columns'])
            df_hist.append(df)
            arguments['start'] += df.shape[0]
            pbar.update(df.shape[0])
            # if arguments['start'] > 300:
            #     break
    return pd.concat(df_hist, axis = 0)

In [ ]:
%%time
groups = [
    'stock_shares', 'currency_selt', 'futures_forts', 'stock_dr', 'stock_ppif', 'stock_index', 'stock_bonds', 
    'stock_etf', 'currency_indices', 'currency_otcindices', 'currency_futures', 'stock_credits', 'stock_deposit',
    'stock_repo_basket', 'money_fk', 'currency_metal','stock_mortgage', 'stock_gcc', 'agro_commodities', 'currency_basket']
filtered_df = []
for grp in groups:
    args = {'start': 0, 'is_trading': 1, 'group_by': 'group', 'group_by_filter': grp}
    tmp = sequential_moex_fetch(args_dict=args)
    filtered_df.append(tmp)
filtered_df = pd.concat(filtered_df, axis = 0)

## api

In [4]:
shares_info_df = pd.read_csv('../data/shares_info.csv', parse_dates=['ipo_date', 'first_1min_candle_date', 'first_1day_candle_date'])
short_list = (
    shares_info_df
    .query('not for_qual_investor_flag and short_enabled_flag and api_trade_available_flag and real_exchange == 1')
    [['figi', 'ticker', 'class_code', 'isin', 'lot', 'currency', 'name', 'exchange']]
)

In [3]:
start_query = make_query(interval=24, start='2015-01-01', end='2024-10-31')

In [4]:
request_url = make_url(engine="stock", market="shares", board="TQBR",security="ABIO", ending = "candles")
request_url

'https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities/ABIO/candles.json'

In [5]:
start = 0
query = {"iss.json": "extended", "iss.meta": "off"} | start_query
async with httpx.AsyncClient() as session:
    query['start'] = start
    # request_url = "https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json"
    data = await get(session, request_url, query)
    # response = await session.get(request_url, params=query)
    # data = response.raise_for_status().json()
    # df = pd.DataFrame(data["securities"]['data'], columns = data["securities"]['columns'])
# df.columns = df.columns.str.lower()

In [7]:
tqbr_shares = pd.read_csv("../data/moex_tqbr_sec.csv")
tqbr_list = tqbr_shares['secid'].tolist()

In [30]:
# df.to_csv("../data/moex_tqbr_sec.csv", index = False)

In [11]:
tqbr_ohlc_df = pd.concat(ohlc_data, axis = 0)
tqbr_ohlc_df.to_csv("../data/moex_tqbr_ohlc.csv", index = False)

In [8]:
ohlc_data = []
async with httpx.AsyncClient() as session:
    for secid in tqbr_shares['secid']:
        print(secid)
        ohlc_tmp = await get_market_candles(session, security = secid, start = '2015-01-01', end = '2024-10-31')
        ohlc_tmp = pd.DataFrame(ohlc_tmp)
        ohlc_tmp['secid'] = secid
        ohlc_data.append(ohlc_tmp)

ABIO
ABRD
ACKO
AFKS
AFLT
AGRO
AKRN
ALRS
AMEZ
APRI
APTK
AQUA
ARSA
ASSB
ASTR
AVAN
BANE
BANEP
BELU
BISVP
BRZL
BSPB
BSPBP
CARM
CBOM
CHGZ
CHKZ
CHMF
CHMK
CIAN
CNTL
CNTLP
DATA
DELI
DIAS
DIOD
DVEC
DZRD
DZRDP
EELT
ELFV
ELMT
ENPG
ETLN
EUTR
FEES
FESH
FIVE
FIXP
FLOT
GAZC
GAZP
GAZS
GAZT
GCHE
GECO
GEMA
GEMC
GLTR
GMKN
HEAD
HIMCP
HNFG
HYDR
IGST
IGSTP
INGR
IRAO
IRKT
IVAT
JNOS
JNOSP
KAZT
KAZTP
KBSB
KCHE
KCHEP
KGKC
KGKCP
KLSB
KLVZ
KMAZ
KMEZ
KOGK
KRKN
KRKNP
KRKOP
KROT
KROTP
KRSB
KRSBP
KUBE
KUZB
KZOS
KZOSP
LEAS
LENT
LKOH
LMBZ
LNZL
LNZLP
LSNG
LSNGP
LSRG
LVHK
MAGE
MAGEP
MAGN
MBNK
MDMG
MFGS
MFGSP
MGKL
MGNT
MGNZ
MGTS
MGTSP
MISB
MISBP
MOEX
MRKC
MRKK
MRKP
MRKS
MRKU
MRKV
MRKY
MRKZ
MRSB
MSNG
MSRS
MSTT
MTLR
MTLRP
MTSS
MVID
NAUK
NFAZ
NKHP
NKNC
NKNCP
NKSH
NLMK
NMTP
NNSB
NNSBP
NSVZ
NVTK
OGKB
OKEY
OMZZP
OZON
OZPH
PHOR
PIKK
PLZL
PMSB
PMSBP
POSI
PRFN
PRMB
PRMD
QIWI
RASP
RDRB
RENI
RGSS
RKKE
RNFT
ROLO
ROSB
ROSN
ROST
RTGZ
RTKM
RTKMP
RTSB
RTSBP
RUAL
RUSI
RZSB
SAGO
SAGOP
SARE
SAREP
SBER
SBERP
SELG
SFIN
SGZH
SIBN
SLEN
SMLT
SNG

## test

In [3]:
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2022-01-01')
async with httpx.AsyncClient() as session:
    request_url = "https://iss.moex.com/iss/engines/stock/markets/shares/boards/TQBR/securities.json"
    moex_info_resp = (await session.get(request_url)).raise_for_status().json()
    moex_info_df = pd.DataFrame(moex_info_resp["securities"]['data'], 
                                columns = moex_info_resp["securities"]['columns'])
    ohlc_data = []
    for secid in moex_info_df['secid']:
        ohlc_tmp = await get_market_candles(session, security = secid, start = start, end = end)
        ohlc_tmp = pd.DataFrame(ohlc_tmp)
        ohlc_tmp['secid'] = secid
        ohlc_data.append(ohlc_tmp)
    moex_candles = pd.concat(ohlc_data, axis = 0)
    moex_candles.begin = moex_candles.begin.map(pd.Timestamp.normalize)
    moex_candles.end = moex_candles.end.map(pd.Timestamp.normalize)

KeyError: 'secid'